Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## Prototype: Deploy Inception Model 

Prototype for Gallery scenario

### Download Inception V3 Models

In [ ]:
import time

start = time.time()

!wget -nc -P model/ http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz
!tar xvzf model/inception-2015-12-05.tgz -C model/
!rm model/cropped_panda.jpg
!rm model/inception-2015-12-05.tgz
    
end = time.time()
print("Model download latency: {} seconds".format(end-start))

### Load AML Workspace

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()
print(ws.name, ws.subscription_id, ws.resource_group, ws.location)

### Register InceptionV3 Models to MMS

It contains multiple models

In [ ]:
from azureml.core.model import Model

models = []

start = time.time()
new_model = Model.register(
    workspace = ws,
    model_path = "model/classify_image_graph_def.pb",
    model_name = "classify_image_graph_def")
models.append(new_model)

end = time.time()
print("Register model latency: {} seconds".format(end-start))
print("Model id={}, name={}, created time={}".format(
    new_model.id, 
    new_model.name, 
    new_model.created_time))

In [ ]:
start = time.time()
new_model = Model.register(
    workspace = ws,
    model_path = "model/imagenet_2012_challenge_label_map_proto.pbtxt",
    model_name = "imagenet_label_map")
models.append(new_model)

end = time.time()
print("Register model latency: {} seconds".format(end-start))
print("Model id={}, name={}, created time={}".format(
    new_model.id, 
    new_model.name, 
    new_model.created_time))

In [ ]:
start = time.time()
new_model = Model.register(
    workspace = ws,
    model_path = "model/imagenet_synset_to_human_label_map.txt",
    model_name = "imagenet_s2h_label_map")
models.append(new_model)

end = time.time()
print("Register model latency: {} seconds".format(end-start))
print("Model id={}, name={}, created time={}".format(
    new_model.id, 
    new_model.name, 
    new_model.created_time))

### Build Container Image 

In [ ]:
from azureml.core.image import Image, ContainerImage

image_config = ContainerImage.image_configuration(
    execution_script = "score.py",
    runtime = "python", 
    dependencies = ["app.py", "ui.html", "inception.py"],
    conda_file = "env.yml")

start = time.time()
image = Image.create(
    workspace = ws,
    name = "inception-v3-demo-image",
    models = models,
    image_config = image_config
)

image.wait_for_creation(show_output = True)

end = time.time()
print("Image creation latency: {} seconds".format(end-start))
print(image)

### Deploy Image to ACI

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice

aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

start = time.time()
aci_service = Webservice.deploy_from_image(
    workspace = ws,
    name = "inception-v3-demo-service",
    image = image,
    deployment_config = aci_config)
aci_service.wait_for_deployment(show_output = True)

end = time.time()
print("Service deployment latency: {} seconds".format(end-start))

### Run Inference

*TODO*: Make it UX friendly via either run a script with sample data or let us consume the UI page


In [ ]:
# Get demo URL 
demo_uri = aci_service.scoring_uri.replace("/score", "/ui")
print("Try out the inceptionv2 model at {}".format(demo_uri))

### Cleanup Resources

Delete the web serivce, image, and models

In [ ]:
aci_service.delete()
image.delete()
for m in models:
    m.delete()
print("Successfully deleted models, image and service.")